In [41]:
def get_time(minute_after_midnight):
    # hh:mm
    minute_after_midnight += 480 # 8:00
    hh = minute_after_midnight // 60
    mm = minute_after_midnight % 60
    return str(hh) + ":" + str(mm).zfill(2)

def time_to_emoji(time_str):
    hour, minute = map(int, time_str.split(":"))
    index = (hour % 12) * 2 + (minute // 30)
    emoji_list = [ "🕛", "🕧", "🕐", "🕜", "🕑", "🕝", "🕒", "🕞", "🕓", "🕟", "🕔", "🕠", "🕕", "🕡", "🕖", "🕢", "🕗", "🕣", "🕘", "🕤", "🕙", "🕥", "🕚", "🕦" ]
    return emoji_list[index]


In [42]:
import import_ipynb
from dijkstra import nodes, arc_matrix, get_shortest_path, format_path
from tsp_costruttivo import build_tsp_path, multi_build_tsp_path

In [43]:
import random
import numpy as np

random.seed(0)

MINUTES_IN_A_DAY = 24*60
MINUTES_PER_WORK_CYCLE = 120
DIMENSIONE_PIASTRA = 94
MASSIMA_CAPACITA_TRASPORTABILE = 6

distribuzione_tamponi_al_minuto = np.array([random.randint(0, 10) for _ in range(MINUTES_IN_A_DAY)])

distribuzione_reparti = np.array([random.random() for _ in range(len(nodes))])
junction_nodes_start_index = nodes.index('X1')
distribuzione_reparti[junction_nodes_start_index:] = 0
# distribuzione_reparti /= distribuzione_reparti.sum()
# print(distribuzione_reparti)

print(distribuzione_tamponi_al_minuto)
print(distribuzione_reparti)

[6 6 0 ... 3 0 5]
[0.5290949  0.94135742 0.68025796 0.630908   0.62781515 0.49698971
 0.73091927 0.24919444 0.89175426 0.27447266 0.94494501 0.92649671
 0.07792452 0.4481797  0.74403628 0.44965407 0.50889902 0.80682394
 0.70499216 0.95800422 0.16448599 0.92355929 0.92798625 0.63474894
 0.94039083 0.25268559 0.88178728 0.77347929 0.609689   0.09062924
 0.03013435 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]


In [44]:
## funzioni di aggiornamento dei tamponi
def aggiornamento_tamponi(tamponi_per_reparto, current_minute):
    return aggiornamento_tamponi_semplice(tamponi_per_reparto, current_minute)

# ad-hoc
def aggiornamento_tamponi_semplice(tamponi_per_reparto, current_minute):
    
    if(current_minute==1):
        reparto = nodes.index('21')
        tamponi_per_reparto[reparto] += 1
        print("🧪➡️🚩Arrivato 1 tampone al reparto 21")
    
    if(current_minute==10):
        reparto = nodes.index('22')
        tamponi_per_reparto[reparto] += 7
        print("🧪➡️🚩 Arrivati 7 tamponi al reparto 22")
    
    if(current_minute>10):
        reparto = random.randint(0, len(nodes)-1)
        qt = random.randint(0, 8)
        if qt > 0:
            tamponi_per_reparto[reparto] += qt
            print(f"🧪➡️🚩 Arrivati {qt} tamponi al reparto {nodes[reparto]}")
    
    return tamponi_per_reparto

# tramite istanza giornaliera
def aggiornamento_tamponi_istanza(tamponi_per_reparto, current_minute):
    # DA IMPLEMENTARE
    return tamponi_per_reparto

# tramite distribuzione statistica
def aggiornamento_tamponi_statistica(tamponi_per_reparto, current_minute):
    nuovi_tamponi_per_reparto = (distribuzione_tamponi_al_minuto[current_minute] * distribuzione_reparti).astype(int)
    tamponi_per_reparto += nuovi_tamponi_per_reparto

    return tamponi_per_reparto


In [45]:
## funzioni di pianificazione in caso di assenza di tamponi
def pianifica_no_tamponi(tamponi_per_reparto, start, current_minute):
    pianifica_rimani_fermo(tamponi_per_reparto, start, current_minute)

# nessun piano
def pianifica_rimani_fermo(tamponi_per_reparto, start, current_minute):
    return None

In [46]:
## funzioni di pianificazione in caso di assenza di tamponi (multiagente)
def multi_pianifica_no_tamponi(env, team_raccolta):
    return multi_pianifica_rimani_fermo(env, team_raccolta)

# nessun piano
def multi_pianifica_rimani_fermo(env, team_raccolta):
    return (None,None)

In [47]:
def get_complete_path(path_grafo_ridotto, current_position):
    path = [current_position]
    for node in path_grafo_ridotto:
        (path_to_node, _) = get_shortest_path(arc_matrix, current_position, node)
        # print("current_position", nodes[current_position])
        # print("destination node", nodes[node])
        # print("path_to_node", format_path(path_to_node))
        path.extend(path_to_node[1:])
        current_position = node
    
    path = path[1:]
    return path

# current_position = nodes.index('X1')
# path = [nodes.index('22'), nodes.index('9')]

# print("Current position:", nodes[current_position])
# print("Path:", format_path(path))

# path_completo = get_complete_path(path, current_position)
# print("Complete path:", format_path(path_completo))

In [48]:
## funzioni di pianificazione (lista di nodi nell'ordine in cui si vogliono visitare)
def pianifica(tamponi_per_reparto, current_position, current_minute, tamponi_trasportati, moving_info):
    # return pianifica_gotomax(tamponi_per_reparto, current_position, current_minute)
    
    if(np.sum(tamponi_per_reparto)==0):
        return pianifica_no_tamponi(tamponi_per_reparto, current_position, current_minute)
    
    current_work_cycle_minute = current_minute % MINUTES_PER_WORK_CYCLE
    residual_time = MINUTES_PER_WORK_CYCLE - current_work_cycle_minute
    path_grafo_ridotto = build_tsp_path(arc_matrix, tamponi_per_reparto, residual_time, tamponi_trasportati, current_position, env.lab, moving_info=moving_info)
    path = get_complete_path(path_grafo_ridotto, current_position)
    return path

# si va al reparto con più tamponi da raccogliere, il piano è calcolato con Dijkstra
def pianifica_gotomax(tamponi_per_reparto, start, current_minute):
    end = np.argmax(tamponi_per_reparto)
    
    if tamponi_per_reparto[end] == 0: # non ci sono tamponi da raccogliere
        return pianifica_no_tamponi(tamponi_per_reparto, start, current_minute)

    (path, distance) = get_shortest_path(arc_matrix, start, end)

    path = path[1:]
    return path

In [49]:
# from pprint import pprint
## funzioni di pianificazione multi-agente
def multi_pianifica(env, team_raccolta):
    
    if(np.sum(env.tamponi_per_reparto)==0):
        return multi_pianifica_no_tamponi(env, team_raccolta)
    
    agent_info = multi_build_tsp_path(arc_matrix, env, team_raccolta)
    # pprint(agent_info)
    paths = {agent: get_complete_path(info['path'],info['current_node']) for agent, info in agent_info.items()}
    # print(paths)
    raccolta = {agent: info['raccolta'] for agent, info in agent_info.items()}
    # print(raccolta)
    return (paths, raccolta)


In [50]:
## funzioni di valutazione dei piani
def get_score(tamponi_per_reparto, current_position, tamponi_trasportati, piano, moving_time):
    return get_score_density(tamponi_per_reparto, current_position, tamponi_trasportati, piano, moving_time)

# score = somma dei tamponi da raccogliere diviso il tempo di percorrenza (tamponi/minuto)
def get_score_density(tamponi_per_reparto, current_position, tamponi_trasportati, piano, moving_time):
    if piano in (None, []):
        return 0

    # calcolo tamponi
    tamponi_nel_path = sum(tamponi_per_reparto[p] for p in piano)
    tamponi_nel_path = min(tamponi_nel_path, MASSIMA_CAPACITA_TRASPORTABILE - tamponi_trasportati)
    
    # calcolo distanza
    distanza = arc_matrix[current_position][piano[0]]
    for i in range(len(piano)-1):
        distanza += arc_matrix[piano[i]][piano[i+1]]
    distanza -= moving_time
    
    # print("tamponi", score, "distanza", distanza)
    score = tamponi_nel_path / distanza
    return score

In [51]:
def reconsider(tamponi_per_reparto, current_position, current_minute, piano, tamponi_trasportati, moving_info):
    (_,_,moving_time) = moving_info
    score = get_score(tamponi_per_reparto, current_position, tamponi_trasportati, piano, moving_time)
    
    piano_alternativo = pianifica(tamponi_per_reparto, current_position, current_minute, tamponi_trasportati, moving_info=moving_info)
    
    if piano_alternativo in (None, []):
        return False, None
    
    moving_time_alternativo = moving_time
    if(piano not in (None, []) and piano[0] != piano_alternativo[0]):
        moving_time_alternativo = -moving_time
    score_alternativo = get_score(tamponi_per_reparto, current_position, tamponi_trasportati, piano_alternativo, moving_time_alternativo)
    
    # if score_alternativo > score:
    #     print(f"piano originale: {format_path(piano)} score: {score}")
    #     print(f"piano alternativo: {format_path(piano_alternativo)} score: {score_alternativo}")
        
    return score_alternativo > score, piano_alternativo

In [52]:
# funzione di riconsiderazione multi-agente
def multi_reconsider(env, team_raccolta):
    
    # calcolo current score
    current_score = 0
    for addetto in team_raccolta:
        (_,_,moving_time) = addetto.moving_info
        current_score += get_score(env.tamponi_per_reparto, addetto.current_position, addetto.tamponi_trasportati, addetto.piano, moving_time)
    #     print(f"{addetto} piano originale: {format_path(addetto.piano)}")
    # print(f"score: {current_score}")
    
    # multi_pianifica(env, team_raccolta)
    alternative_plans_per_agent, raccolta_alternativa = multi_pianifica(env, team_raccolta)
    
    
    if alternative_plans_per_agent in (None, []):
        return False, None, None
        
    # calcolo alternative score
    alternative_score = 0
    for addetto in alternative_plans_per_agent:
        alternative_plan = alternative_plans_per_agent[addetto]
        (_,_,moving_time_alternativo) = addetto.moving_info
        if(alternative_plan not in (None,[]) and addetto.piano not in (None, []) and addetto.piano[0] != alternative_plan[0]):
            moving_time_alternativo = -moving_time
        alternative_score += get_score(env.tamponi_per_reparto, addetto.current_position, addetto.tamponi_trasportati, alternative_plan, moving_time_alternativo)

    #     print(f"{addetto} piano alternativo: {format_path(alternative_plan)}")
    # print(f"alternative score: {alternative_score}")
    
    return alternative_score > current_score, alternative_plans_per_agent, raccolta_alternativa

In [53]:
def move_toward(addetto, next_position):
    moving_info = addetto.moving_info
    current_position = addetto.current_position
    (_,_,moving_time) = moving_info
    distance = int(get_shortest_path(arc_matrix, current_position, next_position)[1])
    moving_time = min(moving_time+1, distance)
    moving_info = (current_position, next_position, moving_time)
    return distance, moving_info

In [54]:
def get_status(env, team_raccolta):
    status = f"{time_to_emoji(get_time(env.current_minute))} Minuto {env.current_minute} ({get_time(env.current_minute)})\n"
    
    if np.sum(env.tamponi_per_reparto) > 0:
        status += f"🏥🧪 {np.sum(env.tamponi_per_reparto)}: "
        for reparto,tamponi in enumerate(env.tamponi_per_reparto):
            if tamponi > 0:
                status += f"🧪 {tamponi} 🚩 {nodes[reparto]}"
                if np.sum(env.tamponi_per_reparto[reparto+1:]) > 0:
                    status += ", "
    else:
        status += "🧪 Nessun tampone da raccogliere"
    
    for addetto in team_raccolta:
        n_tamponi_in_piano = 0
        for reparto in addetto.piano:
            n_tamponi_in_piano += env.tamponi_per_reparto[reparto]
        status += f"\n - {addetto}: 🚩 {nodes[addetto.current_position]} 🪣 {addetto.tamponi_trasportati} 📜 ({n_tamponi_in_piano}🧪) [{format_path(addetto.piano)}]"
    return status


In [55]:
def move_condition(addetto, env, percentage_threshold=1, piano_alternativo=None):
    
    if piano_alternativo is None:
        plan = addetto.piano
    else:
        plan = piano_alternativo
    
    n_tamponi_in_piano = 0
    for reparto in plan:
        n_tamponi_in_piano += env.tamponi_per_reparto[reparto]
    
    n_tamponi_in_piano = min(n_tamponi_in_piano, MASSIMA_CAPACITA_TRASPORTABILE - addetto.tamponi_trasportati)

    return addetto.current_position != env.lab or n_tamponi_in_piano >= percentage_threshold * MASSIMA_CAPACITA_TRASPORTABILE

In [56]:
# multi-agente
def aggiorna_piani(env, team_raccolta):
    must_reconsider, piani_alternativi_per_addetto, raccolta_alternativa = multi_reconsider(env, team_raccolta)
    if must_reconsider:
        for addetto in piani_alternativi_per_addetto:
            piano_alternativo = piani_alternativi_per_addetto[addetto]
            if addetto.piano != piano_alternativo and move_condition(addetto, env, piano_alternativo=piano_alternativo):
                print(f"{addetto}💭 riconsidera il piano {format_path(addetto.piano)} con il piano alternativo {format_path(piano_alternativo)}")
                # considero inversione del percorso su corridoio
                if(piano_alternativo not in (None,[]) and addetto.piano not in (None, []) and addetto.piano[0] != piano_alternativo[0]):
                    (moving_from, moving_toward, moving_time) = addetto.moving_info
                    addetto.moving_info = (moving_toward, moving_from, -moving_time)
                addetto.piano = piano_alternativo
                addetto.raccolta = raccolta_alternativa[addetto]

In [57]:

class Environment:
    current_minute = 0
    tamponi_per_reparto = np.zeros(len(nodes)).astype(int)
    tamponi_in_laboratorio = 0
    tamponi_in_processamento = 0
    tamponi_processati = 0
    
    lab = nodes.index('7')
    
    def __init__(self):
        pass
    
    def get_residual_time(self):
        current_work_cycle_minute = env.current_minute % MINUTES_PER_WORK_CYCLE
        residual_time = MINUTES_PER_WORK_CYCLE - current_work_cycle_minute
        return residual_time
    
    # aggiornamento_tamponi
    
env = Environment()

In [58]:

class Agent:
    id = None
    current_position = None
    tamponi_trasportati = 0
    moving_info = (None, None, 0)
    piano = []
    
    def __init__(self, id, env):
        self.id = id
        self.current_position = env.lab
        
    def __str__(self):
        return f"{self.id}"
    
    def __repr__(self):
        return f"{self.id}"
    

In [59]:
team_raccolta = [ Agent("👨🏻‍⚕️",env), Agent("👨🏽‍⚕️",env), Agent("👩‍⚕️",env) ]

In [60]:
last_minute = MINUTES_IN_A_DAY # debug

while env.current_minute < MINUTES_IN_A_DAY:
    print(get_status(env, team_raccolta))
    
    if env.get_residual_time() == MINUTES_PER_WORK_CYCLE: # macchinario partito
        if env.tamponi_in_processamento > 0:
            env.tamponi_processati += env.tamponi_in_processamento
            print(f"✨ Fine ciclo di lavoro ({env.tamponi_processati}🧪)")
        env.tamponi_in_processamento = min(env.tamponi_in_laboratorio, DIMENSIONE_PIASTRA)
        env.tamponi_in_laboratorio -= env.tamponi_in_processamento
        print(f"🚀 Inizio ciclo di lavoro ({env.tamponi_in_processamento}🧪)")
    
    env.tamponi_per_reparto = aggiornamento_tamponi(env.tamponi_per_reparto, env.current_minute)
        
    aggiorna_piani(env, team_raccolta)
    
    for addetto in team_raccolta:
        if addetto.piano not in [[], None] and move_condition(addetto, env):
            # print(f"{addetto}📜: {nodes[addetto.current_position]} -> {format_path(addetto.piano)}")
            
            next_position = addetto.piano[0]
            distance, addetto.moving_info = move_toward(addetto, next_position)
            # print(moving_info)
            (_,_,moving_time) = addetto.moving_info
            
            if distance != moving_time:
                print(f"{addetto}🔜🚩 si sta spostando da {nodes[addetto.current_position]} a {nodes[next_position]} ... {moving_time}/{distance}")
            else: # arrivato a destinazione
                print(f"{addetto}➡️🚩 arrivato al nodo {nodes[next_position]}")
                addetto.current_position = next_position
                addetto.piano = addetto.piano[1:]
                addetto.moving_info = (None, None, 0)
                
                # raccolta tamponi (old)
                if env.tamponi_per_reparto[addetto.current_position] > 0 and min(env.tamponi_per_reparto[addetto.current_position], MASSIMA_CAPACITA_TRASPORTABILE - addetto.tamponi_trasportati) > 0:
                    tamponi_raccolti = min(env.tamponi_per_reparto[addetto.current_position], MASSIMA_CAPACITA_TRASPORTABILE - addetto.tamponi_trasportati)
                    print(f"{addetto}🧪🪣 Tamponi raccolti: {tamponi_raccolti}")
                    addetto.tamponi_trasportati += tamponi_raccolti
                    env.tamponi_per_reparto[addetto.current_position] -= tamponi_raccolti
                
                # # raccolta tamponi (test)
                # if addetto.current_position in addetto.raccolta and env.tamponi_per_reparto[addetto.current_position] > 0 and min(env.tamponi_per_reparto[addetto.current_position], MASSIMA_CAPACITA_TRASPORTABILE - addetto.tamponi_trasportati) > 0:
                #     tamponi_raccolti = addetto.raccolta[addetto.current_position]
                #     print(f"{addetto}🧪🪣 Tamponi raccolti: {tamponi_raccolti}")
                #     addetto.tamponi_trasportati += tamponi_raccolti
                #     env.tamponi_per_reparto[addetto.current_position] -= tamponi_raccolti
                #     # addetto.raccolta.pop(addetto.current_position)
                
                # arrivo al laboratorio
                if addetto.current_position == env.lab:
                    env.tamponi_in_laboratorio += addetto.tamponi_trasportati
                    print(f"{addetto}🪣🏥 ha depositato in laboratorio: {tamponi_raccolti}")
                    addetto.tamponi_trasportati = 0
    
    if env.tamponi_in_laboratorio > 0:
        print(f"🏭🧪 Tamponi totali in laboratorio: {env.tamponi_in_laboratorio}")
    if env.tamponi_processati > 0:
        print(f"✨🧪 Tamponi totali processati: {env.tamponi_processati}")
    
    env.current_minute += 1
    
    # print(tamponi_per_reparto)    
            
    ## TESTING
    if env.current_minute > last_minute:
        break
    ##

    print()

🕗 Minuto 0 (8:00)
🧪 Nessun tampone da raccogliere
 - 👨🏻‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👨🏽‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👩‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
🚀 Inizio ciclo di lavoro (0🧪)

🕗 Minuto 1 (8:01)
🧪 Nessun tampone da raccogliere
 - 👨🏻‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👨🏽‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👩‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
🧪➡️🚩Arrivato 1 tampone al reparto 21

🕗 Minuto 2 (8:02)
🏥🧪 1: 🧪 1 🚩 21
 - 👨🏻‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👨🏽‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👩‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]

🕗 Minuto 3 (8:03)
🏥🧪 1: 🧪 1 🚩 21
 - 👨🏻‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👨🏽‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👩‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]

🕗 Minuto 4 (8:04)
🏥🧪 1: 🧪 1 🚩 21
 - 👨🏻‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👨🏽‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👩‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]

🕗 Minuto 5 (8:05)
🏥🧪 1: 🧪 1 🚩 21
 - 👨🏻‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👨🏽‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 👩‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]

🕗 Minuto 6 (8:06)
🏥🧪 1: 🧪 1 🚩 21
 - 👨🏻‍⚕️: 🚩 7 🪣 0 📜 (0🧪) [No path]
 - 